In [1]:
import numpy as np
import torch
import local_utils
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from local_utils import ResidualBlock
from torch import nn
import cv2
from pathlib import Path
from collections import OrderedDict
from types import SimpleNamespace
import torch.quantization


### W tej części ćwiczenia, wczytamy nauczony model zmiennoprzecinkowy MiniResNet, skwantyzujemy go do postaci stałoprzecinkowej i na koniec skompilujemy go.

# 1. Dane ewaluacyjne

Zaczynamy od ponownego stworzenia generatora danych na bazie danych MNIST:

Wystarczy nam sama część testowa. Ustawiamy `batch_size` na 1.

In [2]:
class SuperPointDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_paths = [str(f) for f in Path(image_folder).iterdir() if f.suffix == ".ppm"]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx], cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (300, 200))
        if self.transform:
            image = self.transform(image)
        return image, self.image_paths[idx]

transform = transforms.Compose([transforms.ToTensor()])

test_dataset = SuperPointDataset("./data", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(f"Liczba obrazów w zbiorze testowym: {len(test_dataset)}")


Liczba obrazów w zbiorze testowym: 6


Dodatkowo przygotujemy plik w formacie `.npz` na podstawie danych testowych. Wykorzystamy go do ewaluacji modelu na docelowej platformie Kria.

Uzupełnij wektory `quantization_data` oraz `quantization_labels` danymi z `test_loadera`. Wykorzystaj do tego pętle `for` oraz `.append` (Przykład wykorzystania DataLoader'a z pętlą `for` przedstawiono w 1 części podczas wczytywania danych).

Następnie każdy wektor z osobna połącz funkcją `torch.cat` z parametrem `dim=0` i przekonwertuj je do formatu `ndarray` za pomocą `.numpy()`. 

Zapisz je funkcją np.savez.

In [3]:
#quantization_data = []
#quantization_labels = []

#TODO
#Fill quantization vectors

#train_X = ... #TODO
#train_Y = ... #TODO

#np.savez('eval_MNIST.npz', data=..., targets=...) #TODO

#print('Done')

# 2. Inicjalizacja modelu zmiennoprzecinkowego

Tworzymy taką samą klasę sieci Superpoint jak w pierwszej części ćwiczenia.

In [4]:
def sample_descriptors(keypoints, descriptors, s: int = 8):
    """Interpolate descriptors at keypoint locations"""
    b, c, h, w = descriptors.shape
    keypoints = (keypoints + 0.5) / (keypoints.new_tensor([w, h]) * s)
    keypoints = keypoints * 2 - 1  # normalize to (-1, 1)
    descriptors = torch.nn.functional.grid_sample(
        descriptors, keypoints.view(b, 1, -1, 2), mode="bilinear", align_corners=False
    )
    descriptors = torch.nn.functional.normalize(
        descriptors.reshape(b, c, -1), p=2, dim=1
    )
    return descriptors


def batched_nms(scores, nms_radius: int):
    assert nms_radius >= 0

    def max_pool(x):
        return torch.nn.functional.max_pool2d(
            x, kernel_size=nms_radius * 2 + 1, stride=1, padding=nms_radius
        )

    zeros = torch.zeros_like(scores)
    max_mask = scores == max_pool(scores)
    for _ in range(2):
        supp_mask = max_pool(max_mask.float()) > 0
        supp_scores = torch.where(supp_mask, zeros, scores)
        new_max_mask = supp_scores == max_pool(supp_scores)
        max_mask = max_mask | (new_max_mask & (~supp_mask))
    return torch.where(max_mask, scores, zeros)


def select_top_k_keypoints(keypoints, scores, k):
    if k >= len(keypoints):
        return keypoints, scores
    scores, indices = torch.topk(scores, k, dim=0, sorted=True)
    return keypoints[indices], scores


class VGGBlock(nn.Sequential):
    def __init__(self, c_in, c_out, kernel_size, relu=True):
        padding = (kernel_size - 1) // 2
        conv = nn.Conv2d(
            c_in, c_out, kernel_size=kernel_size, stride=1, padding=padding
        )
        activation = nn.ReLU(inplace=True) if relu else nn.Identity()
        bn = nn.BatchNorm2d(c_out, eps=0.001)
        super().__init__(
            OrderedDict(
                [
                    ("conv", conv),
                    ("activation", activation),
                    ("bn", bn),
                ]
            )
        )


class SuperPoint(nn.Module):
    default_conf = {
        "nms_radius": 4,
        "max_num_keypoints": 500,
        "detection_threshold": 0.005,
        "remove_borders": 4,
        "descriptor_dim": 256,
        "channels": [64, 64, 128, 128, 256],
    }

    def __init__(self, **conf):
        super().__init__()
        conf = {**self.default_conf, **conf}
        self.conf = SimpleNamespace(**conf)
        self.stride = 2 ** (len(self.conf.channels) - 2)
        channels = [1, *self.conf.channels[:-1]]

        backbone = []
        for i, c in enumerate(channels[1:], 1):
            layers = [VGGBlock(channels[i - 1], c, 3), VGGBlock(c, c, 3)]
            if i < len(channels) - 1:
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            backbone.append(nn.Sequential(*layers))
        self.backbone = nn.Sequential(*backbone)

        c = self.conf.channels[-1]
        self.detector = nn.Sequential(
            VGGBlock(channels[-1], c, 3),
            VGGBlock(c, self.stride**2 + 1, 1, relu=False),
        )
        self.descriptor = nn.Sequential(
            VGGBlock(channels[-1], c, 3),
            VGGBlock(c, self.conf.descriptor_dim, 1, relu=False),
        )

    def forward(self, data):
        image = data
        if image.shape[1] == 3:  # RGB to gray
            scale = image.new_tensor([0.299, 0.587, 0.114]).view(1, 3, 1, 1)
            image = (image * scale).sum(1, keepdim=True)

        features = self.backbone(image)
        descriptors_dense = torch.nn.functional.normalize(
            self.descriptor(features), p=2, dim=1
        )

        # Decode the detection scores
        scores = self.detector(features)

        scores = torch.nn.functional.softmax(scores, 1)[:, :-1]

        b, _, h, w = scores.shape
        scores = scores.permute(0, 2, 3, 1).reshape(b, h, w, self.stride, self.stride)
        scores = scores.permute(0, 1, 3, 2, 4).reshape(
            b, h * self.stride, w * self.stride
        )

        scores = batched_nms(scores, self.conf.nms_radius)
       # lol = scores.numpy()
       
        # Discard keypoints near the image borders
        if self.conf.remove_borders:
            pad = self.conf.remove_borders
            scores[:, :pad] = -1
            scores[:, :, :pad] = -1
            scores[:, -pad:] = -1
            scores[:, :, -pad:] = -1

        # Extract keypoints
        if b > 1:
            idxs = torch.where(scores > self.conf.detection_threshold)
            mask = idxs[0] == torch.arange(b, device=scores.device)[:, None]
        else:  # Faster shortcut
            scores = scores.squeeze(0)
            idxs = torch.where(scores > self.conf.detection_threshold)

        # Convert (i, j) to (x, y)
        keypoints_all = torch.stack(idxs[-2:], dim=-1).flip(1).float()
        scores_all = scores[idxs]

        keypoints = []
        scores = []
        descriptors = []
        for i in range(b):
            if b > 1:
                k = keypoints_all[mask[i]]
                s = scores_all[mask[i]]
            else:
                k = keypoints_all
                s = scores_all
            if self.conf.max_num_keypoints is not None:
                k, s = select_top_k_keypoints(k, s, self.conf.max_num_keypoints)
            d = sample_descriptors(k[None], descriptors_dense[i, None], self.stride)
            keypoints.append(k)
            scores.append(s)
            descriptors.append(d.squeeze(0).transpose(0, 1))

        return {
            "keypoints": keypoints,
            "keypoint_scores": scores,
            "descriptors": descriptors,
        }

    
class SuperPoint_short(nn.Module):
    default_conf = {
        "nms_radius": 4,
        "max_num_keypoints": 500,
        "detection_threshold": 0.005,
        "remove_borders": 4,
        "descriptor_dim": 256,
        "channels": [64, 64, 128, 128, 256],
    }

    def __init__(self, **conf):
        super().__init__()
        conf = {**self.default_conf, **conf}
        self.conf = conf
        self.stride = 2 ** (len(self.conf["channels"]) - 2)
        channels = [1, *self.conf["channels"][:-1]]

        # Definicja QuantStub
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

        backbone = []
        for i, c in enumerate(channels[1:], 1):
            layers = [VGGBlock(channels[i - 1], c, 3), VGGBlock(c, c, 3)]
            if i < len(channels) - 1:
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            backbone.append(nn.Sequential(*layers))
        self.backbone = nn.Sequential(*backbone)

        c = self.conf["channels"][-1]
        self.detector = nn.Sequential(
            VGGBlock(channels[-1], c, 3),
            VGGBlock(c, self.stride**2 + 1, 1, relu=False),
        )
        self.descriptor = nn.Sequential(
            VGGBlock(channels[-1], c, 3),
            VGGBlock(c, self.conf["descriptor_dim"], 1, relu=False),
        )

    def forward(self, data):
        # Kwantyzacja wejścia
        image = self.quant(data)

        features = self.backbone(image)
        descriptors_dense = torch.nn.functional.normalize(
            self.descriptor(features), p=2, dim=1
        )

        # Decode the detection scores
        scores = self.detector(features)

        scores = torch.nn.functional.softmax(scores, 1)[:, :-1]

        b, _, h, w = scores.shape
        scores = scores.permute(0, 2, 3, 1).reshape(b, h, w, self.stride, self.stride)
        scores = scores.permute(0, 1, 3, 2, 4).reshape(
            b, h * self.stride, w * self.stride
        )

        # Dekwantyzacja przed wyjściem
        return self.dequant(scores), self.dequant(descriptors_dense)


Tworzymy model i wgrywamy wagi z pliku MNIST.pth. Zapisujemy go do urządzenia (w dockerze dostępny jest tylko CPU!) i ustawiamy go na ewaluację `.eval()`.

In [5]:
device = torch.device("cpu")
detection_thresh = 0.005
nms_radius = 5

float_model = SuperPoint_short(detection_threshold=detection_thresh, nms_radius=nms_radius).eval()
float_model.load_state_dict(torch.load("model_weights_legacy.pth"))

input_shape = (1, 1, 200, 300)
model = float_model.to(device)

print(device)

cpu


# 3. Ewaluacja modelu zmiennoprzecinkowego

Przed przystąpieniem do kwantyzacji, wykonamy szybką ewaluację modelu zmiennoprzecinkowego. Sprawdzimy, czy dane są dobrze przygotowane i czy model został odpowiednio zapisany i wczytany. Wczytujemy metrykę Accuracy z `local_utils`.

In [6]:
def evaluate_orig(model,
             dataloader):
    tm = local_utils.TimeMeasurement("Evaluation", len(dataloader))
    with torch.no_grad(), tm:
        score = 0.0
        cntr = 0
        for i, XY in enumerate(dataloader):
            X = XY[0]
            Y = XY[1:]
            y_pred = model(X)
            score = score*cntr + X.shape[0]*evaluator(y_pred, *Y)
            cntr += X.shape[0]
            score /= cntr
            print("\rEvaluation {}/{}. Score = {}".format(i,len(dataloader), score),end='')
        
        print("\rEvaluation {}/{}. Score = {}".format(len(dataloader),len(dataloader), score),end='\n')
    print(tm)

In [7]:
import torch
import torch.nn.functional as F

def evaluate(model, dataloader, device):
    """
    Porównuje działanie modelu przed i po kwantyzacji.

    :param float_model: model przed kwantyzacją
    :param quantized_model: model po kwantyzacji
    :param dataloader: zbiór testowy
    :param device: urządzenie (CPU/GPU)
    """
    tm = local_utils.TimeMeasurement("Evaluation", len(dataloader))
    
    model.to(device).eval()

    mae_total, mse_total, count = 0.0, 0.0, 0

    with torch.no_grad(), tm:
        for i, (X, *Y) in enumerate(dataloader):
            X = X.to(device)
            # Forward pass dla modeli
            scores, descriptors_dense = model(X)
            torch.save({'scores': scores, 'descriptors_dense': descriptors_dense}, f'outputs_batch_{i}.pt')
            data = torch.load(f'data/outputs_batch_{i}.pt')

            # Obliczenie różnicy
            mae = torch.mean(torch.abs(scores - data['scores'])).item()
            mse = F.mse_loss(scores, data['scores']).item()
            
            mae += torch.mean(torch.abs(descriptors_dense - data['descriptors_dense'])).item()
            mse += F.mse_loss(descriptors_dense, data['descriptors_dense']).item()
            
            # Aktualizacja wyników
            batch_size = X.shape[0]
            mae_total += mae * batch_size
            mse_total += mse * batch_size
            count += batch_size

            print(f"\rEvaluation {i+1}/{len(dataloader)} | MAE: {mae:.6f} | MSE: {mse:.6f}", end='')

    # Średnie wyniki
    mae_avg = mae_total / count
    mse_avg = mse_total / count

    print(f"\nFinal Evaluation: MAE = {mae_avg:.6f}, MSE = {mse_avg:.6f}")
    print(tm)

    return mae_avg, mse_avg


In [8]:
# You can evaluate your floating point model first 
evaluate(model, test_loader, device)

Evaluation 6/6 | MAE: 0.000000 | MSE: 0.000000
Final Evaluation: MAE = 0.000000, MSE = 0.000000
Execution time: 0.0:0.0:0:840, processed 6 frames, throughput: 7.13833969895999 fps.


(0.0, 0.0)

#### Jeżeli wszystko działa poprawnie, a uzyskana dokładność jest na podobnym poziome jak podczas uczenia, możemy przejść do kwantyzacji.

# 4. Kwantyzacja modelu zmiennoprzecinkowego

### Vitis AI Quantizer dla Post Training Quantization składa się z dwóch części.
Pierwszą częścią jest kalibracja (mode='calib') - Vitis AI Quantizer analizuje model i dostosowuje parametry kwantyzacji.
 
Drugą częścią jest ewaluacja/testowanie (mode='test') - sprawdzana jest dokładność modelu (nie powinna być duża zmiana) i model jest eksportowany do formatu .xmodel.

### Do obu części wykorzystamy funkcję quantize.

Funkcja wykorzystuje kwantyzator dla PyTorch z gita Vitis AI: https://github.com/Xilinx/Vitis-AI/tree/1.4/tools/Vitis-AI-Quantizer/vai_q_pytorch

In [9]:
def quantize(float_model:torch.nn.Module, 
             input_shape:tuple,
             quant_dir:str, 
             quant_mode:str, 
             device:torch.device,
             dataloader):
    """
    :param float_model: float model with loaded weights
    :param input_shape: shape of input(CH,W,H)
    :param quant_dir: path to directory with quantized model components
    :param quant_mode: quant_mode in ['calib', 'test'] 
    :param data_loader: data_loader - for 'calib' must be batch_size == 1
    :param evaluator: fcn/obj like: fcn(y_pred, y_ref) -> float 
    """
    tm = local_utils.TimeMeasurement("Quantization", len(dataloader))
    from pytorch_nndct.apis import torch_quantizer, dump_xmodel

    with tm:
        # model to device
        model = float_model.to(device)

        # Force to merge BN with CONV for better quantization accuracy
        optimize = 1

        rand_in = torch.randn(input_shape)
        print("get qunatizer start")
        try:
            quantizer = torch_quantizer(
                quant_mode, model, rand_in, output_dir=quant_dir, device=device)
        except Exception as e:
            print("exception:")
            print(e)
            return
        print("get qunatizer end")

        print("get quantized model start")
        quantized_model = quantizer.quant_model
        print("get quantized model end")

        # evaluate
        print("testing start")
        mae, mse = evaluate(quantized_model, dataloader, device)
        print(f"Testing finished: MAE = {mae:.6f}, MSE = {mse:.6f}")
        print("testing end")

        # export config
        if quant_mode == 'calib':
            print("export config")
            quantizer.export_quant_config()
            print("export config end")
        # export model
        if quant_mode == 'test':
            print("export xmodel")
            quantizer.export_xmodel(deploy_check=False, output_dir=quant_dir)
            print("export xmodel end")
    print(tm)

Zaczynamy od kalibracji. Jako wejście funkcji podajemy:
- float_model - model zmiennoprzecinkowy, który uzyskaliśmy podczas uczenia,
- input_shape - wymiar danych wejściowych w formacie [batch, CH, W, H],
- quant_dir - folder do którego zostanie zapisany wynik kwantyzacji,
- quant_mode - do wyboru 'calib' albo 'test',
- device - urządzenie na którym zostaną wykonane obliczenia (CPU),
- dataloader - dane na którym będą wykonane obliczenia,
- evaluator - metryka, względem której będzie sprawdzana dokładność

### Uwaga! Kwantyzacja w procesie kalibracji jest wolna. W przypadku dużych modeli i dużych wymiarów danych, nie można przesadzić z ilością danych.

Po udanej kalibracji, czas na testowanie i zapisanie modelu. Uruchamiamy funkcję ze zmienionym parametrem mode na 'test'.

Proces ten jest szybszy od kalibracji.

In [10]:
# Quantize model - test - is faster

#TODO
quantize(float_model=model, 
         input_shape=input_shape,
         quant_dir='quant_dir', # directory for quantizer results
         quant_mode='calib',
         device=device,
         dataloader=test_loader)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

[VAIQ_NOTE]: Loading NNDCT kernels...
get qunatizer start

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing SuperPoint_short...

[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir/SuperPoint_short.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.
get quantized model end
testing start


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

Po testowaniu, należy skompilować model. Podajemy odpowiednio parametry:

- --xmodel - ścieżka do zapisanego modelu (zapisany jest w folderze podanym podczas kwantyzacji jako parametr 'quant_dir'
- --arch - podajemy plik arch.json, który znajdował się w pliku. Jest to numer (fingerprint), który określa typ DPU sprzętu docelowego
- --net_name - nazwa naszego modelu po kompilacji - dowolna
- --output_dir - folder do którego zapisany zostanie model

In [ ]:
# compile model
!vai_c_xir --xmodel 'quant_dir/MiniResNet_int.xmodel' --arch arch.json --net_name MiniResNet_qu --output_dir build

Teraz przejdziemy do testowania modelu na sprzęcie docelowym.

#### Wersja 1: Podłączenie do sieci
Zanim podłączysz zasilanie do Kria, podepnij kabel USB do komputera, a kabel Ethernetowy do sieci, w której znajduje się komputer.

Podłącz Kria do zasilania i poczekaj minutę, aż system się uruchomi.

Uruchom `cutecom` z `sudo`. Włącz port odpowiadający do Kria. Jeżeli pojawi się napis `kria login:` zaloguj się:

`login: ubuntu`

`hasło: Xilinx123`

Po zalogowaniu się, powinny pojawić się informacje systemowe. Nas interesuje adres `IPv4` dla `eth0`. Skopiuj go i dodaj do niego `:9090` - przykładowa wartość `192.168.1.26:9090`. Wklej to w przeglądarce. Powinien pojawić się kolejny Jupyter. Zaloguj się do niego hasłem:

`xilinx`

#### Wersja 2: Podłączenie do komputera
Zanim podłączysz zasilanie do Kria, podepnij kabel USB do komputera oraz kabel Ethernetowy pomiędzy Kria a PC. Na PC włącz ustawienia sieci `Wired Setting` -> `IPv4` -> `Shared to other computers`. Włącz zasilanie płytki.
Uruchom `cutecom` z `sudo`. Włącz port odpowiadający do Kria. Jeżeli pojawi się napis `kria login:` zaloguj się:

`login: ubuntu`

`hasło: Xilinx123`

Po zalogowaniu się, powinny pojawić się informacje systemowe. Nas interesuje adres `inet` dla `eth0`. Skopiuj go i dodaj do niego `:9090` - przykładowa wartość `10.42.0.47:9090`. Wklej to w przeglądarce. Powinien pojawić się kolejny Jupyter. Zaloguj się do niego hasłem:

`xilinx`

#### Przesyłanie plików
Stwórz nowy folder i nazwij go `PSRA_Lab`. Przenieś do niego odpowiednio pliki:
- dpu.bit, 
- dpu.hwh, 
- dpu.xclbin, 
- eval_MNIST.npz lub tak jak nazwałeś swój plik z danymi do ewaluacji
- MiniResNet_compiled.xmodel lub tak jak nazwałeś swój skompilowany plik

Można to wykonać komendą `scp`, ale łatwiej jest przeciągnąć pliki z folderu do Jupyter Notebook'a.

# UWAGA! 

Jeżeli wystąpi problem z adresami IPv4 Kria (po podłaczeniu kilku płytek do jednej sieci, każda z nich będzie miała taki sam adres), należy sprawdzić, czy działa komenda w konsoli `cutecom`:

`ifconfig`

Jeżeli nie, to należy zainstalować poprzez:

 `sudo apt install net-tools`.  
 
Po tym należy odpiąć kabel Ethernet z Kria, poczekać kilka sekund i wpisać w konsole `cutecom`:

`hostname -I`

Jeżeli konsola nie zwróci żadnego błędu oraz żadnego aresu IP to wpisz w konsole `cutecom`:

`sudo ifconfig eth0 192.168.1.x netmask 255.255.255.0`

Tutaj podany adres powinien być taki sam jak przykładowa wartość wyżej. Ustawiamy wartość `x` na inną niż była np. 123. Chcemy uniknąć konfliktu pomiędzy płytkami ale również komputerami. Po tym znowu ponawiamy:

`hostname -I`.

Powinien pojawić sie ustawiony przez nas adres. Podpinamy kabel Ethernet i uruchamiamy w przeglądarce Jupyter z ustawionym adresem IP.